In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
#import sagemaker
#import boto3
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns',None) # To display all columns
pd.set_option('display.max_rows',None) # To display all columns
#client = boto3.client('s3')
import feather
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from functools import reduce
from datetime import datetime
from dateutil.relativedelta import relativedelta
import io
import sys

In [2]:
df = pd.read_feather('Dataframe27.ftr')
#dfq=pd.read_feather('dfq.ftr')

In [3]:
df['bundle_id'] ='b' + df['bundle_id'].astype(str)

In [ ]:
df_aux = df[['user_iD','bundle_id','correct', 'timestamp']].sort_values(by=['timestamp']) 

# hard coded new headers (possible to build from real 'bundle_id's)
df_new_headers = ['b{}'.format(x+1) for x in range(8933)]
df_new_headers.insert(0, 'user_iD')

dict_answered = {}
# create a new dataframe (I'm sure that there is a more Pythonish solution)
df_new_data = []
user_ids = sorted(set( [x for label, x in df_aux.user_iD.items()]))
for user_id in user_ids:
    dict_answered[user_id] = 0
    if len( sys.argv) > 1 and sys.argv[1]:
        # supplied arg in the next line for better result readability
        df_new_values = [sys.argv[1].strip('"').strip("'")
            for x in range(len(df_new_headers)-1)]
    else:
        # zeroes (original assignment)
        df_new_values = [0 for x in range(len(df_new_headers)-1)]
    
    df_new_values.insert(0, user_id)
    df_new_data.append(df_new_values)

df_new = pd.DataFrame(data=df_new_data, columns=df_new_headers)

# fill the new dataframe using values from the original one
for aux in df_aux.itertuples(index=True, name=None):
    if aux[3] == 1.0:
        # add 1 to number of already answered questions for current user 
        dict_answered[aux[1]] += 1
        df_new.loc[ df_new["user_iD"] == aux[1], aux[2]] = dict_answered[aux[1]]    

#print( df_new)

In [ ]:
df_new.head()

In [ ]:
df_new.to_feather('Out.ftr')